In [ ]:
import torch 
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR 

import numpy as np
import os
import sys
import json

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
from tqdm import tqdm


# pv_vision_dir = os.path.join(Path.home(), 'pv-vision')
pv_vision_dir = os.path.join('/home/eccoope', 'pv-vision')
# functions_dir = os.path.join(Path.home(), 'el_img_cracks_ec', 'scripts')
functions_dir = os.path.join('/home/eccoope', 'el_img_cracks_ec', 'scripts')

sys.path.append(pv_vision_dir)
sys.path.append(functions_dir)

# ojas_functions_dir = os.path.join(Path.home(), 'pvcracks/retrain/')
ojas_functions_dir = '/Users/ojas/Desktop/saj/SANDIA/pvcracks/retrain/'
sys.path.append(ojas_functions_dir)

from tutorials.unet_model import construct_unet
import functions

In [ ]:
# root = Path('/projects/wg-psel-ml/EL_images/osanghi/CornersIHDEANE/')
# root = Path('/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/CornersIHDEANE/')
# root = Path('/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Norman_LineCorners/')
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/LineCorners_With_Original/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/CWRU_SunEdison_Mono/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/CWRU_Dupont_Mono/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Combined_CWRU_SunEdison_and_Dupont_Mono/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/LBNL_Mono_Cells/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Combined_CWRU_LBNL_ASU/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/ASU_IHDEANE/"

root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_Combined_CWRU_LBNL_ASU/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_CWRU_SunEdison/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_ASU/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_LBNL/"

model_weight_paths = {
    "emma_retrained": "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/retrained_pv-vision_model.pt",
    "original": "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/pv-vision_model.pt",
}

# weight_path = model_weight_paths["emma_retrained"]
weight_path = model_weight_paths["original"]

# checkpoint_name = "line_corners_og_weights_checkpoint"
# checkpoint_name = "CWRU_SunEdison_Mono"
# checkpoint_name = "CWRU_Dupont_Mono"
# checkpoint_name = "Combined_CWRU_SunEdison_and_Dupont_Mono"
# checkpoint_name = "LBNL_Mono_Cells"
# checkpoint_name = "Combined_CWRU_LBNL_ASU"

checkpoint_name = "Fresh_Combined_CWRU_LBNL_ASU"
# checkpoint_name = "Fresh_CWRU_SunEdison"
# checkpoint_name = "Fresh_ASU"
# checkpoint_name = "Fresh_LBNL"

In [ ]:
def load_dataset(root):
    transformers = functions.Compose([functions.FixResize(256), functions.ToTensor(), functions.Normalize()])
    
    train_dataset = functions.SolarDataset(root, image_folder="img/train", 
            mask_folder="ann/train", transforms=transformers)
    
    val_dataset = functions.SolarDataset(root, image_folder="img/val", 
            mask_folder="ann/val", transforms=transformers)

    return train_dataset, val_dataset

In [ ]:
def load_device_and_model(weight_path):
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device("mps")
    unet = construct_unet(5)
    unet = torch.nn.DataParallel(unet)
    
    checkpoint = torch.load(weight_path, map_location=device)
    from collections import OrderedDict

    new_state_dict = OrderedDict()
    if weight_path == model_weight_paths["emma_retrained"]:
        for k, v in checkpoint.items():
            name = "module." + k
            new_state_dict[name] = v
    elif weight_path == model_weight_paths["original"]:
        for k, v in checkpoint.items():
            new_state_dict[k] = v
    
    unet.load_state_dict(new_state_dict)

    model = unet.module.to(device)
    
    return device, model

In [ ]:
from pathlib import Path

def get_save_dir(base_dir, checkpoint_name):
    checkpoint_dir = base_dir + "/checkpoints/"
    checkpoint_dir_path = Path(checkpoint_dir)
    folders = [folder.name for folder in checkpoint_dir_path.iterdir() if folder.is_dir()]    
    
    max_number = 0
    for folder in folders:
        number = int(folder[-1])
        if number > max_number:
            max_number = number

    new_folder_name = f"{checkpoint_name}{max_number + 1}"
    new_folder_path = os.path.join(checkpoint_dir, new_folder_name)
    
    os.makedirs(new_folder_path, exist_ok=True)
    
    return new_folder_path

In [ ]:
train_dataset, val_dataset = load_dataset(root)
device, model = load_device_and_model(weight_path)

In [ ]:
batch_size_val=1
batch_size_train=1
lr = 0.000001
step_size=1
gamma = 0.1
num_epochs = 30
criterion = torch.nn.SmoothL1Loss()

save_dir = get_save_dir(str(root), checkpoint_name)
os.makedirs(save_dir, exist_ok=True)

params_dict = {'batch_size_val' : batch_size_val,
           'batch_size_train' : batch_size_train,
           'lr' : lr,
           'step_size' : step_size,
           'gamma' : gamma,
           'num_epochs' : num_epochs,
           'criterion' : str(criterion)}

with open(os.path.join(save_dir, 'params.json'), 'w', encoding='utf-8') as f:
    json.dump(params_dict, f, ensure_ascii=False, indent=4)


train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=False)

In [ ]:
optimizer = Adam(model.parameters(), lr=lr)
lr_scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
evaluate_metric=None
running_record = {'train': {'loss': []}, 'val': {'loss': []}}

save_name='model.pt'
cache_output = True

In [ ]:
# category_mapping = {0: "empty", 1: "dark", 2: "busbar", 3: "crack", 4: "corner"}
# category_mapping = {0: "empty", 1: "dark", 2: "busbar", 3: "crack", 4: "non-cell area"}
category_mapping = {0: "empty", 1: "dark", 2: "busbar", 3: "crack", 4: "non-cell"}

In [ ]:
def inference_and_show(idx, retrained=False):
    img, mask = train_loader.dataset. __getitem__(idx)
    img = img.to(device)
    raw_img, _ = train_loader.dataset. __getraw__(idx)
    test_res = model(img.unsqueeze(0)).detach().cpu().numpy().squeeze()#.argmax(axis = 0)
    test_res = np.argmax(test_res, axis = 0)

    mask_cpu = mask.cpu().numpy()

    cmap = mpl.colormaps['viridis'].resampled(5)  # define the colormap
    cmaplist = [cmap(i) for i in range(5)]

    fig, ax = plt.subplots(ncols=3, figsize=(12,12))

    im = ax[0].imshow(raw_img.convert('L'), cmap='gray', interpolation='None')
    ax[0].axis('off')

    clim = (0, 4)
    im = ax[1].imshow(mask_cpu, cmap = 'viridis', clim=clim)
    ax[1].axis('off')
    ax[1].set_title("Ground Truth Mask")

    ax[2].imshow(test_res, cmap = 'viridis', clim = clim, interpolation='None')
    handles, labels = ax[2].get_legend_handles_labels()

    for c, classlabel in zip(cmaplist, [f'({k}) {v}' for k, v in category_mapping.items()]):
            patch = mpatches.Patch(color=c, label=classlabel, ec='k')
            handles.append(patch)
    ax[2].legend(handles=handles, fontsize='x-small')
    ax[2].axis('off')
    if retrained:
        ax[2].set_title("Retrained Model Prediction")
    else:
        ax[2].set_title("Model Prediction")

# Training

In [ ]:
training_epoch_loss = []
val_epoch_loss = []

for epoch in tqdm(range(1, num_epochs + 1)):
    
    training_step_loss = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        output_activated = 1 /(1 + torch.exp(-output))
        target_onehot = torch.nn.functional.one_hot(target, num_classes = 5)
        target_onehot = torch.moveaxis(target_onehot, 3, 1)
        
        training_loss = criterion(output_activated, target_onehot.float())
        training_loss.backward()
        optimizer.step()
        training_step_loss.append(training_loss.item())
        
    training_epoch_loss.append(np.array(training_step_loss).mean())
        
    val_step_loss = []
    
    for batch_idx, (data, target) in enumerate(val_loader):
        data, target = data.to(device), target.to(device)
        data = data.to(device)
        
        output = model(data)
        output_activated = 1 /(1 + torch.exp(-output))
        target_onehot = torch.nn.functional.one_hot(target, num_classes = 5)
        target_onehot = torch.moveaxis(target_onehot, 3, 1)
        val_loss = criterion(output_activated, target_onehot.float())
        
        val_step_loss.append(val_loss.item())
        
    val_epoch_loss.append(np.array(val_step_loss).mean())
    
    os.makedirs(os.path.join(save_dir, f'epoch_{epoch}'), exist_ok=True)
    torch.save(model.state_dict(), os.path.join(save_dir, f'epoch_{epoch}', save_name))
    print(f'Saved model at epoch {epoch}')
    
    if epoch >= 2 and epoch < 30:
        os.remove(os.path.join(save_dir, f'epoch_{epoch-1}', save_name))
        print(f'Removed model at epoch {epoch-1}')

In [ ]:
inference_and_show(-32, retrained=True)

In [ ]:
inference_and_show(13, retrained=True)

In [ ]:
inference_and_show(44, retrained=True)

In [ ]:
inference_and_show(1, retrained=True)

In [ ]:
fig, ax = plt.subplots()

x = np.arange(1, len(training_epoch_loss) + 1, 1)

ax.scatter(x, training_epoch_loss, label='training loss')
ax.scatter(x, val_epoch_loss, label='validation loss')
ax.legend()
ax.set_xlabel('Epoch')

print(training_epoch_loss)


In [ ]:
val_epoch_loss

# SECOND TRAINING

In [ ]:
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_Combined_CWRU_LBNL_ASU/"
root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_CWRU_SunEdison/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_ASU/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_LBNL/"

# checkpoint_name = "Fresh_Combined_CWRU_LBNL_ASU"
checkpoint_name = "Fresh_CWRU_SunEdison"
# checkpoint_name = "Fresh_ASU"
# checkpoint_name = "Fresh_LBNL"

In [ ]:
train_dataset, val_dataset = load_dataset(root)
device, model = load_device_and_model(weight_path)

In [ ]:
batch_size_val=1
batch_size_train=1
lr = 0.000001
step_size=1
gamma = 0.1
num_epochs = 30
criterion = torch.nn.SmoothL1Loss()

save_dir = get_save_dir(str(root), checkpoint_name)
os.makedirs(save_dir, exist_ok=True)

params_dict = {'batch_size_val' : batch_size_val,
           'batch_size_train' : batch_size_train,
           'lr' : lr,
           'step_size' : step_size,
           'gamma' : gamma,
           'num_epochs' : num_epochs,
           'criterion' : str(criterion)}

with open(os.path.join(save_dir, 'params.json'), 'w', encoding='utf-8') as f:
    json.dump(params_dict, f, ensure_ascii=False, indent=4)


train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=False)

In [ ]:
optimizer = Adam(model.parameters(), lr=lr)
lr_scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
evaluate_metric=None
running_record = {'train': {'loss': []}, 'val': {'loss': []}}

save_name='model.pt'
cache_output = True

In [ ]:

training_epoch_loss = []
val_epoch_loss = []

for epoch in tqdm(range(1, num_epochs + 1)):
    
    training_step_loss = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        output_activated = 1 /(1 + torch.exp(-output))
        target_onehot = torch.nn.functional.one_hot(target, num_classes = 5)
        target_onehot = torch.moveaxis(target_onehot, 3, 1)
        
        training_loss = criterion(output_activated, target_onehot.float())
        training_loss.backward()
        optimizer.step()
        training_step_loss.append(training_loss.item())
        
    training_epoch_loss.append(np.array(training_step_loss).mean())
        
    val_step_loss = []
    
    for batch_idx, (data, target) in enumerate(val_loader):
        data, target = data.to(device), target.to(device)
        data = data.to(device)
        
        output = model(data)
        output_activated = 1 /(1 + torch.exp(-output))
        target_onehot = torch.nn.functional.one_hot(target, num_classes = 5)
        target_onehot = torch.moveaxis(target_onehot, 3, 1)
        val_loss = criterion(output_activated, target_onehot.float())
        
        val_step_loss.append(val_loss.item())
        
    val_epoch_loss.append(np.array(val_step_loss).mean())
    
    os.makedirs(os.path.join(save_dir, f'epoch_{epoch}'), exist_ok=True)
    torch.save(model.state_dict(), os.path.join(save_dir, f'epoch_{epoch}', save_name))
    print(f'Saved model at epoch {epoch}')
    
    if epoch >= 2 and epoch < 30:
        os.remove(os.path.join(save_dir, f'epoch_{epoch-1}', save_name))
        print(f'Removed model at epoch {epoch-1}')

In [ ]:
inference_and_show(-32, retrained=True)

In [ ]:
inference_and_show(13, retrained=True)

In [ ]:
inference_and_show(44, retrained=True)

In [ ]:
inference_and_show(1, retrained=True)

In [ ]:
fig, ax = plt.subplots()

x = np.arange(1, len(training_epoch_loss) + 1, 1)

ax.scatter(x, training_epoch_loss, label='training loss')
ax.scatter(x, val_epoch_loss, label='validation loss')
ax.legend()
ax.set_xlabel('Epoch')

print(training_epoch_loss)


In [ ]:
val_epoch_loss

## THIRD TRAINING

In [ ]:
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_Combined_CWRU_LBNL_ASU/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_CWRU_SunEdison/"
root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_ASU/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_LBNL/"

# checkpoint_name = "Fresh_Combined_CWRU_LBNL_ASU"
# checkpoint_name = "Fresh_CWRU_SunEdison"
checkpoint_name = "Fresh_ASU"
# checkpoint_name = "Fresh_LBNL"

In [ ]:
train_dataset, val_dataset = load_dataset(root)
device, model = load_device_and_model(weight_path)

In [ ]:
batch_size_val=1
batch_size_train=1
lr = 0.000001
step_size=1
gamma = 0.1
num_epochs = 30
criterion = torch.nn.SmoothL1Loss()

save_dir = get_save_dir(str(root), checkpoint_name)
os.makedirs(save_dir, exist_ok=True)

params_dict = {'batch_size_val' : batch_size_val,
           'batch_size_train' : batch_size_train,
           'lr' : lr,
           'step_size' : step_size,
           'gamma' : gamma,
           'num_epochs' : num_epochs,
           'criterion' : str(criterion)}

with open(os.path.join(save_dir, 'params.json'), 'w', encoding='utf-8') as f:
    json.dump(params_dict, f, ensure_ascii=False, indent=4)


train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=False)

In [ ]:
optimizer = Adam(model.parameters(), lr=lr)
lr_scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
evaluate_metric=None
running_record = {'train': {'loss': []}, 'val': {'loss': []}}

save_name='model.pt'
cache_output = True

In [ ]:

training_epoch_loss = []
val_epoch_loss = []

for epoch in tqdm(range(1, num_epochs + 1)):
    
    training_step_loss = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        output_activated = 1 /(1 + torch.exp(-output))
        target_onehot = torch.nn.functional.one_hot(target, num_classes = 5)
        target_onehot = torch.moveaxis(target_onehot, 3, 1)
        
        training_loss = criterion(output_activated, target_onehot.float())
        training_loss.backward()
        optimizer.step()
        training_step_loss.append(training_loss.item())
        
    training_epoch_loss.append(np.array(training_step_loss).mean())
        
    val_step_loss = []
    
    for batch_idx, (data, target) in enumerate(val_loader):
        data, target = data.to(device), target.to(device)
        data = data.to(device)
        
        output = model(data)
        output_activated = 1 /(1 + torch.exp(-output))
        target_onehot = torch.nn.functional.one_hot(target, num_classes = 5)
        target_onehot = torch.moveaxis(target_onehot, 3, 1)
        val_loss = criterion(output_activated, target_onehot.float())
        
        val_step_loss.append(val_loss.item())
        
    val_epoch_loss.append(np.array(val_step_loss).mean())
    
    os.makedirs(os.path.join(save_dir, f'epoch_{epoch}'), exist_ok=True)
    torch.save(model.state_dict(), os.path.join(save_dir, f'epoch_{epoch}', save_name))
    print(f'Saved model at epoch {epoch}')
    
    if epoch >= 2 and epoch < 30:
        os.remove(os.path.join(save_dir, f'epoch_{epoch-1}', save_name))
        print(f'Removed model at epoch {epoch-1}')

In [ ]:
inference_and_show(-32, retrained=True)

In [ ]:
inference_and_show(13, retrained=True)

In [ ]:
inference_and_show(44, retrained=True)

In [ ]:
inference_and_show(1, retrained=True)

In [ ]:
fig, ax = plt.subplots()

x = np.arange(1, len(training_epoch_loss) + 1, 1)

ax.scatter(x, training_epoch_loss, label='training loss')
ax.scatter(x, val_epoch_loss, label='validation loss')
ax.legend()
ax.set_xlabel('Epoch')

print(training_epoch_loss)


In [ ]:
val_epoch_loss

## FOURTH TRAINING

In [ ]:
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_Combined_CWRU_LBNL_ASU/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_CWRU_SunEdison/"
# root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_ASU/"
root = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Fresh_LBNL/"

# checkpoint_name = "Fresh_Combined_CWRU_LBNL_ASU"
# checkpoint_name = "Fresh_CWRU_SunEdison"
# checkpoint_name = "Fresh_ASU"
checkpoint_name = "Fresh_LBNL"

In [ ]:
train_dataset, val_dataset = load_dataset(root)
device, model = load_device_and_model(weight_path)

In [ ]:
batch_size_val=1
batch_size_train=1
lr = 0.000001
step_size=1
gamma = 0.1
num_epochs = 30
criterion = torch.nn.SmoothL1Loss()

save_dir = get_save_dir(str(root), checkpoint_name)
os.makedirs(save_dir, exist_ok=True)

params_dict = {'batch_size_val' : batch_size_val,
           'batch_size_train' : batch_size_train,
           'lr' : lr,
           'step_size' : step_size,
           'gamma' : gamma,
           'num_epochs' : num_epochs,
           'criterion' : str(criterion)}

with open(os.path.join(save_dir, 'params.json'), 'w', encoding='utf-8') as f:
    json.dump(params_dict, f, ensure_ascii=False, indent=4)


train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=False)

In [ ]:
optimizer = Adam(model.parameters(), lr=lr)
lr_scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
evaluate_metric=None
running_record = {'train': {'loss': []}, 'val': {'loss': []}}

save_name='model.pt'
cache_output = True

In [ ]:

training_epoch_loss = []
val_epoch_loss = []

for epoch in tqdm(range(1, num_epochs + 1)):
    
    training_step_loss = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        output_activated = 1 /(1 + torch.exp(-output))
        target_onehot = torch.nn.functional.one_hot(target, num_classes = 5)
        target_onehot = torch.moveaxis(target_onehot, 3, 1)
        
        training_loss = criterion(output_activated, target_onehot.float())
        training_loss.backward()
        optimizer.step()
        training_step_loss.append(training_loss.item())
        
    training_epoch_loss.append(np.array(training_step_loss).mean())
        
    val_step_loss = []
    
    for batch_idx, (data, target) in enumerate(val_loader):
        data, target = data.to(device), target.to(device)
        data = data.to(device)
        
        output = model(data)
        output_activated = 1 /(1 + torch.exp(-output))
        target_onehot = torch.nn.functional.one_hot(target, num_classes = 5)
        target_onehot = torch.moveaxis(target_onehot, 3, 1)
        val_loss = criterion(output_activated, target_onehot.float())
        
        val_step_loss.append(val_loss.item())
        
    val_epoch_loss.append(np.array(val_step_loss).mean())
    
    os.makedirs(os.path.join(save_dir, f'epoch_{epoch}'), exist_ok=True)
    torch.save(model.state_dict(), os.path.join(save_dir, f'epoch_{epoch}', save_name))
    print(f'Saved model at epoch {epoch}')
    
    if epoch >= 2 and epoch < 30:
        os.remove(os.path.join(save_dir, f'epoch_{epoch-1}', save_name))
        print(f'Removed model at epoch {epoch-1}')

In [ ]:
inference_and_show(-32, retrained=True)

In [ ]:
inference_and_show(13, retrained=True)

In [ ]:
inference_and_show(44, retrained=True)

In [ ]:
inference_and_show(1, retrained=True)

In [ ]:
fig, ax = plt.subplots()

x = np.arange(1, len(training_epoch_loss) + 1, 1)

ax.scatter(x, training_epoch_loss, label='training loss')
ax.scatter(x, val_epoch_loss, label='validation loss')
ax.legend()
ax.set_xlabel('Epoch')

print(training_epoch_loss)


In [ ]:
val_epoch_loss